In [8]:
import json
import pandas as pd

with open('../data/orgs/uk/govuk_orgs_enriched.json', 'r') as file:
    json_data = json.load(file)


# Read JSON data into a DataFrame
df = pd.read_json('../data/orgs/uk/govuk_orgs_enriched.json')

# Write DataFrame to CSV
df.to_csv('data.csv', index=False)

In [2]:
# test data
print(json.dumps(json_data, indent=4))

[
    {
        "id": "https://www.gov.uk/api/organisations/united-kingdom-national-nuclear-laboratory",
        "title": "United Kingdom National Nuclear Laboratory",
        "format": "Public corporation",
        "updated_at": "2024-12-19T10:34:45.000+00:00",
        "web_url": "https://www.gov.uk/government/organisations/united-kingdom-national-nuclear-laboratory",
        "details": {
            "slug": "united-kingdom-national-nuclear-laboratory",
            "abbreviation": "UKNNL",
            "logo_formatted_name": "United Kingdom\r\nNational Nuclear\r\nLaboratory",
            "organisation_brand_colour_class_name": "department-for-business-innovation-skills",
            "organisation_logo_type_class_name": "single-identity",
            "closed_at": null,
            "govuk_status": "exempt",
            "govuk_closed_status": null,
            "content_id": "78c48d24-7f78-4591-a221-b36c2aa1e151"
        },
        "analytics_identifier": "PC1435",
        "parent_organisa

In [19]:
df

,id,title,format,updated_at,web_url,details,analytics_identifier,parent_organisations,child_organisations,superseded_organisations,superseding_organisations,non_govuk_domain
0,https://www.gov.uk/api/organisations/united-ki...,United Kingdom National Nuclear Laboratory,Public corporation,2024-12-19T10:34:45.000+00:00,https://www.gov.uk/government/organisations/un...,{'slug': 'united-kingdom-national-nuclear-labo...,PC1435,[],[],[],[],https://uknnl.com/
1,https://www.gov.uk/api/organisations/valuation...,Valuation Tribunal for England,Tribunal,2024-07-16T13:58:40.000+01:00,https://www.gov.uk/government/organisations/va...,"{'slug': 'valuation-tribunal-for-england', 'ab...",PB1204,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],https://www.valuationtribunal.gov.uk/about-us/vte
2,https://www.gov.uk/api/organisations/valuation...,Valuation Tribunal Service,Executive non-departmental public body,2024-07-16T13:57:03.000+01:00,https://www.gov.uk/government/organisations/va...,{'slug': 'valuation-tribunal-service-for-engla...,PB375,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.valuationtribunal.gov.uk/
3,https://www.gov.uk/api/organisations/vehicle-c...,Vehicle Certification Agency,Executive agency,2018-09-21T10:04:59.000+01:00,https://www.gov.uk/government/organisations/ve...,"{'slug': 'vehicle-certification-agency', 'abbr...",EA80,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],https://www.vehicle-certification-agency.gov.uk/
4,https://www.gov.uk/api/organisations/victims-c...,Victims' Commissioner,Other,2014-12-03T09:56:05.000+00:00,https://www.gov.uk/government/organisations/vi...,"{'slug': 'victims-commissioner', 'abbreviation...",PB326,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://victimscommissioner.org.uk/
5,https://www.gov.uk/api/organisations/victoria-...,Victoria and Albert Museum,Executive non-departmental public body,2023-02-14T09:59:09.000+00:00,https://www.gov.uk/government/organisations/vi...,"{'slug': 'victoria-and-albert-museum', 'abbrev...",PB378,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.vam.ac.uk
6,https://www.gov.uk/api/organisations/visitbritain,VisitBritain,Executive non-departmental public body,2023-02-14T09:59:29.000+00:00,https://www.gov.uk/government/organisations/vi...,"{'slug': 'visitbritain', 'abbreviation': 'VB',...",PB186,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.visitbritain.org/
7,https://www.gov.uk/api/organisations/visitengland,VisitEngland,Executive non-departmental public body,2015-05-18T09:43:22.000+01:00,https://www.gov.uk/government/organisations/vi...,"{'slug': 'visitengland', 'abbreviation': 'VE',...",PB376,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.visitengland.com
8,https://www.gov.uk/api/organisations/wales-aud...,Wales Audit Office,Other,2015-04-01T18:56:43.000+01:00,https://www.gov.uk/government/organisations/wa...,"{'slug': 'wales-audit-office', 'abbreviation':...",OT908,[],[],[],[],http://www.wao.gov.uk/
9,https://www.gov.uk/api/organisations/wallace-c...,Wallace Collection,Executive non-departmental public body,2023-02-14T10:00:04.000+00:00,https://www.gov.uk/government/organisations/wa...,"{'slug': 'wallace-collection', 'abbreviation':...",PB187,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.wallacecollection.org/


In [69]:
df.parent_organisations[1][0].get('id')

'https://www.gov.uk/api/organisations/ministry-of-housing-communities-local-government'

In [71]:
df['id'][0]

'https://www.gov.uk/api/organisations/united-kingdom-national-nuclear-laboratory'

# break down parent-child relationships to allow graphing

In [ ]:
# iot graph parent/child relationships, we want to:
# - break down the nested dict within 'parent' into 'parent_id' and 'parent_url' 
# --- actually we don't need parent_url as they should map
# - so append a new column to the df with parent_id and use that and id to generate the edges


# iterate through each row
# extract parent_id
# create a new df
# zip the two together
df['parent_id'] = df['parent_organisations'].apply(
      lambda x: x[0]['id'] if x and len(x) > 0 else None
)
df


    #print(row.parent_organisations[:].get('id'))
    # print(json_normalize(row.parent_organisations))

,id,title,format,updated_at,web_url,details,analytics_identifier,parent_organisations,child_organisations,superseded_organisations,superseding_organisations,non_govuk_domain,parent_id
0,https://www.gov.uk/api/organisations/united-ki...,United Kingdom National Nuclear Laboratory,Public corporation,2024-12-19T10:34:45.000+00:00,https://www.gov.uk/government/organisations/un...,{'slug': 'united-kingdom-national-nuclear-labo...,PC1435,[],[],[],[],https://uknnl.com/,n/a
1,https://www.gov.uk/api/organisations/valuation...,Valuation Tribunal for England,Tribunal,2024-07-16T13:58:40.000+01:00,https://www.gov.uk/government/organisations/va...,"{'slug': 'valuation-tribunal-for-england', 'ab...",PB1204,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],https://www.valuationtribunal.gov.uk/about-us/vte,https://www.gov.uk/api/organisations/ministry-...
2,https://www.gov.uk/api/organisations/valuation...,Valuation Tribunal Service,Executive non-departmental public body,2024-07-16T13:57:03.000+01:00,https://www.gov.uk/government/organisations/va...,{'slug': 'valuation-tribunal-service-for-engla...,PB375,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.valuationtribunal.gov.uk/,https://www.gov.uk/api/organisations/ministry-...
3,https://www.gov.uk/api/organisations/vehicle-c...,Vehicle Certification Agency,Executive agency,2018-09-21T10:04:59.000+01:00,https://www.gov.uk/government/organisations/ve...,"{'slug': 'vehicle-certification-agency', 'abbr...",EA80,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],https://www.vehicle-certification-agency.gov.uk/,https://www.gov.uk/api/organisations/departmen...
4,https://www.gov.uk/api/organisations/victims-c...,Victims' Commissioner,Other,2014-12-03T09:56:05.000+00:00,https://www.gov.uk/government/organisations/vi...,"{'slug': 'victims-commissioner', 'abbreviation...",PB326,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://victimscommissioner.org.uk/,https://www.gov.uk/api/organisations/ministry-...
5,https://www.gov.uk/api/organisations/victoria-...,Victoria and Albert Museum,Executive non-departmental public body,2023-02-14T09:59:09.000+00:00,https://www.gov.uk/government/organisations/vi...,"{'slug': 'victoria-and-albert-museum', 'abbrev...",PB378,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.vam.ac.uk,https://www.gov.uk/api/organisations/departmen...
6,https://www.gov.uk/api/organisations/visitbritain,VisitBritain,Executive non-departmental public body,2023-02-14T09:59:29.000+00:00,https://www.gov.uk/government/organisations/vi...,"{'slug': 'visitbritain', 'abbreviation': 'VB',...",PB186,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.visitbritain.org/,https://www.gov.uk/api/organisations/departmen...
7,https://www.gov.uk/api/organisations/visitengland,VisitEngland,Executive non-departmental public body,2015-05-18T09:43:22.000+01:00,https://www.gov.uk/government/organisations/vi...,"{'slug': 'visitengland', 'abbreviation': 'VE',...",PB376,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.visitengland.com,https://www.gov.uk/api/organisations/departmen...
8,https://www.gov.uk/api/organisations/wales-aud...,Wales Audit Office,Other,2015-04-01T18:56:43.000+01:00,https://www.gov.uk/government/organisations/wa...,"{'slug': 'wales-audit-office', 'abbreviation':...",OT908,[],[],[],[],http://www.wao.gov.uk/,n/a
9,https://www.gov.uk/api/organisations/wallace-c...,Wallace Collection,Executive non-departmental public body,2023-02-14T10:00:04.000+00:00,https://www.gov.uk/government/organisations/wa...,"{'slug': 'wallace-collection', 'abbreviation':...",PB187,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.wallacecollection.org/,https://www.gov.uk/api/organisations/departmen...


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Create directed graph
g = nx.DiGraph()

# Add edges for rows that have a parent_id (not null)
edges = df[df['parent_id'].notna()][['parent_id', 'id']].to_records(index=False)
g.add_edges_from(edges)

# Add isolated nodes for rows without a parent_id
isolated_nodes = df[df['parent_id'].isna()]['id'].tolist()
g.add_nodes_from(isolated_nodes)

# Create visualization
plt.figure(figsize=(20, 16))

# Use hierarchical layout for better visualization
pos = nx.spring_layout(g, k=2, iterations=50)

# Draw the graph
nx.draw(g, pos, 
        with_labels=False,  # Don't show full URLs as labels
        node_color='lightblue',
        node_size=100,
        edge_color='gray',
        arrows=True,
        arrowsize=10,
        font_size=8)

# Optionally add labels with shortened names
labels = {node: node.split('/')[-1] for node in g.nodes()}
nx.draw_networkx_labels(g, pos, labels, font_size=6)

plt.title(f"UK Government Organizations Hierarchy\n{g.number_of_nodes()} nodes, {g.number_of_edges()} edges")
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"Total nodes: {g.number_of_nodes()}")
print(f"Total edges: {g.number_of_edges()}")
print(f"Isolated nodes (no parent): {len(list(nx.isolates(g)))}")